In [30]:
#importing the libraries and declaring some global variables
import pandas as  pd
import numpy as np
import re

bagOfWords = []
unique_words = set() # the entire vocabluary
uw0 = set() # class 0 vocabluary
uw1 = set() # class 1 vocabluary
freq0 ={} # dictionary to count the occurence of words in class 0 reviews
freq1 ={} # dictionary to count the occurence of words in class 0 reviews
p_c0,p_c1 =0,0 # prior probability
n0,n1 = 0 ,0  # number of class 0 and class 1 reviews
d0,d1,d2 =  pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
d3,d4,D =pd.DataFrame(),pd.DataFrame(),pd.DataFrame()# folds and mterics
acc,fscores,datafolds = [],[],[]

In [31]:
def build_model(train):
    global bagOfWords,unique_words,p_c0,p_c1,n0,n1,freq0,freq1,uw0,uw1
    uw0 = set()
    uw1 = set()
    p_c0,p_c1 =0,0
    n0,n1 = 0 ,0

    bagOfWords = [] # each preprocessed training review as an array of words 
    for ind ,row in train.iterrows():
        bag = [w for w in row['Review'].split() if w!=' '] 
        bagOfWords.append(bag)
        for w in bag:
            if row['Label']=='0':
                uw0.add(w) # building the vocabulary for class 0
            else:
                uw1.add(w) # building the vocabulary for class 1
       
    n0 = len(train.loc[train['Label']=='0'])
    n1 = len(train.loc[train['Label']=='1'])
   
    # print(n0,n1) # no of class 0  and class 1 review
    unique_words = set()
    unique_words = uw0.union(uw1)
   # print(len(unique_words)) # voacbulary size
    freqvect = [] # arr of dicts that store the occurence of each word in a review
    for i in range(len(bagOfWords)):
        d = {}
        #d = dict.fromkeys(unique_words,0)
        for word in bagOfWords[i]:
            if word in d:
                d[word]+=1 #d[word]+=1
            else:
                d[word]=1
        freqvect.append(d)
    
    freq0 = {} # occurence of each word that are present in reviews for which class==0
    freq1 = {}  # occurence of each word that are present in reviews for which class==1
    p_c0,p_c1 =0,0
    for word in unique_words:
        c0,c1 = 0,0
        for i in range(train.shape[0]):
            if word in freqvect[i]:
                if train.iloc[i,1]=='0':
                    c0+=freqvect[i][word]
                    #p_c0+=1
                else:
                    c1+=freqvect[i][word]
                #p_c1+=1
        freq0[word]=c0+1 # adding 1 to occurence of each word to implement laplacian smoothening
        freq1[word]=c1+1 

In [32]:
def predict(test):
    global freq0,freq1,unique_words,n0,n1,uw0,uw1
    
    bowtest = [] # each preprocessed testing review as an array of words
    for ind ,row in test.iterrows():
        bag = [w for w in row['Review'].split() if w!=' '] 
        bowtest.append(bag)
    
    probtest0 = [] # to store prob that ith review belong to class 0
    probtest1 = [] # to store prob that ith review belong to class 1
    p_c0 = n0/(n0+n1) # prior prob of class 0
    p_c1 = n1/(n0+n1) # prior prob of class 1
    for i in range(len(bowtest)):
        p0,p1=1,1
        for word in bowtest[i]:
            if word in unique_words:
                p0*=(freq0[word]/(len(uw0)+len(unique_words))) # +2 adding 10 to the denom because of laplacian smoothening
                p1*=(freq1[word]/(len(uw1)+len(unique_words)))
            else:
                p0*=(1/(len(uw0)+len(unique_words)))
                p1*=(1/(len(uw1)+len(unique_words)))
        probtest0.append(p0*p_c0)
        probtest1.append(p1*p_c1)
    
    pred = [] # storing the predicted labels
    for i,j in zip(probtest0,probtest1):
        if(i>j):
            pred.append(0)
        else:
            pred.append(1)
    
    actu = np.asarray([int(s) for s in test.iloc[:,1]])
    pred = np.asarray(pred)
    y_actu = pd.Series(actu, name='Actual')
    y_pred = pd.Series(pred, name='Predicted')
    # making the comfusion matrix
    cm = pd.crosstab(y_actu, y_pred)
    
    acc = (cm.iloc[0,0]+cm.iloc[1,1])/len(test) # accuracy
    p = cm.iloc[0,0]/(cm.iloc[0,0]+cm.iloc[1,0]) # precision
    r = cm.iloc[0,0]/(cm.iloc[0,0]+cm.iloc[0,1]) # recall
    f_score = 2*p*r/(p+r)
    return (acc,round(f_score,4))


In [33]:
def reshape_data(lines):
    linesasarr=[] # to store the review and label as an array
    for line in lines:
        arr = [str(word) for word in line.split()] # splitting each word
        line=''
        for i in range(len(arr)-1): # the last element is the label so leaving it out from the review
            line+=arr[i]+" "
        line+=','+arr[len(arr)-1] # appending the label to the review separated by a comma
        linesasarr.append(line)
    lines = []
    # separating the review and the label and appending it to a dataframe
    for line in linesasarr:
        ind = line.rindex(',')
        review,label = line[0:ind],line[ind+1:]
        arr= [review,label]
        lines.append(arr)
    data= pd.DataFrame(lines,columns=['Review','Label'])
    return data

In [34]:
def preprocess(unclean):
    for ind ,row in unclean.iterrows():
        review = row['Review'].lower() # to lower case
        filtered_review = ''
        for word in review.split():
            if not word in stopwords: # removing stopwords
                filtered_review+=word+" "
        review = filtered_review
        filtered_review=''
        for word in review.split():
            alpha_only =  re.sub('[^a-zA-Z]','' ,word) # removing symbols
            if alpha_only!=' ' and not alpha_only in stopwords: # double checking for stopwords
                filtered_review+= alpha_only+" "
        row['Review'] = filtered_review.strip() # removing unwanted spaces
    return unclean

In [42]:
def create_folds(data):
    global d0,d1,d2,d3,d4,D,datafolds
    d0,d1,d2,d3,d4,D = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    # using pandas to create 5 folds of the entire data set
    d0 = data.sample(frac=0.2,random_state=29)
    D = data.drop(d0.index)
    d1 = D.sample(frac=0.25,random_state=19)
    D = D.drop(d1.index)
    d2 = D.sample(frac=1/3,random_state=21)
    D = D.drop(d2.index)
    d3 = D.sample(frac=0.5,random_state=260)
    D = D.drop(d3.index)
    d4 = D
    datafolds= [d0,d1,d2,d3,d4]

In [43]:
def run5folds():
    global d0,d1,d2,d3,d4 , acc,fscores,datafolds
    acc =[] # array to store accuracy and fscore of each iteration
    fscores = []
    for i in range(len(datafolds)):
        train = pd.DataFrame()
        test = pd.DataFrame()
        for j in range(len(datafolds)):
            if j!=i:
                train =  train.append(datafolds[j]) # building the training data using 4 folds and leaving 1 out
            else:
                test = datafolds[j] # the leftout set for testing
        #print(len(train),len(test))
        build_model(train)
        a,f = predict(test)
        print('acc = ',a,' fscore = ',f)

        #print(a,f)
        acc.append(a)
        fscores.append(f)
        
    mean = np.mean(acc)
    std = np.std(acc)
    print("\nAccuracy " , round(mean,4)," +- ",round(std,4))
    mean = np.mean(fscores)
    std = np.std(fscores)
    print("FScore ",round(mean,4)," +- ",round(std,4))

In [44]:
f = open('a1_d3.txt','r') # reading the dataset i.e a txt file
lines = f.readlines() # each line is a review along with its label 
f.close()
rawdata = reshape_data(lines) 
f = open('stopwords.txt') # created a textfile containing 191 stopwords
lines = f.readlines()
stopwords = set()
f.close()
for line in lines:
    stopwords.add(str(line)[:-1])

data = rawdata
print('Data not preprocessed \n')
create_folds(data)
run5folds()

print('\nData preprocessed \n')
data = preprocess(rawdata)
create_folds(data)
run5folds()

Data not preprocessed 

acc =  0.755  fscore =  0.7435
acc =  0.745  fscore =  0.7151
acc =  0.715  fscore =  0.6885
acc =  0.805  fscore =  0.8
acc =  0.715  fscore =  0.6667

Accuracy  0.747  +-  0.0331
FScore  0.7228  +-  0.0464

Data preprocessed 

acc =  0.815  fscore =  0.7886
acc =  0.755  fscore =  0.6879
acc =  0.745  fscore =  0.7182
acc =  0.775  fscore =  0.7486
acc =  0.75  fscore =  0.7059

Accuracy  0.768  +-  0.0256
FScore  0.7298  +-  0.0354
